In [1]:
from ipynb.fs.full.Funciones_basicas import *

## greedyAppW

- input: Grafo (G), lista de nodos visitados (V_), cantidad de nodos (N), Cantidad de muestras (K), límite de nodos en la solución (limit) y límite de personas en la solución (wlim)
- output: Lista con las K soluciones

In [ ]:
def greedyAppW(G, V_, N, K, limit, wlim):
    
    V = V_.copy()
    #V = V_
    
    E = []
    for u in range(N):
        if not V[u]:
            E.append((get_size(G, V, u), u))

    S = set()
    for e in E:
        if e[0] > 0 and e[0] <= limit:
            S.add(e[1])

    ans = []
    count = 0
    while count < K:
        maxv = 0; u = -1
        for v in S:
            s, w = get_size_weight(G, V, N, v)
            if s > maxv and w <= wlim:
                u = v
                maxv = s
        if u == -1:
            break

        V, _ = visit(G, V, u)
        ans.append(u)
        S.remove(u)
        count += 1

    return ans

## greedyAppWeightReduce

- input: Grafo (G), Lista de pesos (W), lista de nodos visitados (V_), Cantidad de muestras (K), límite de nodos en la solución (limit) y límite de personas en la solución (wlim)
- output: Lista con las K soluciones, Fracción de nodos visitados

In [4]:
def greedyAppWeightReduce(G, W, V_, K, limit, wlim):
    
    V = V_.copy(); N = len(V)
    
    E = []
    for u in range(N):
        if not V[u]:
            E.append((get_size(G, V, u), u))

    limit_set = set()
    for e in E:
        if e[0] > 0 and e[0] <= limit:
            limit_set.add(e[1])

    ans = []
    while len(ans) < K:
        
        S = [0 for u in range(N)]
        for v in limit_set:
            _, S[v] = get_size_weight(G, W, V, v)
        
        maxv = 0; u = -1
        for v in limit_set:
            s = get_size_precalc(G, S, V, v)
            if s > maxv and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break

        V, _ = visit(G, V, u)
        ans.append(u)
        limit_set.remove(u)
        
    return ans, sum(V) / N


## particionar
- Input: Grafo (G), lista de pesos (W), lista de visitados (V), nodo (node)
- Output: nodo que particiona el ideal

## partition_ideal
- Input: Grafo (G), lista de pesos (W), lista de visitados (V), nodo (node)
- Output: nodo que particiona el ideal y su ideal inducido

In [ ]:
def particionar(G, W, V_, node):
    I = set(get_ideal(G, V_, node)); N = len(V_)
    V = [0 if i in I else 1 for i in range(N)]

    R = get_size(G, V, node)
    low = 0; high = R
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppWeightReduce(G, W, V, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppWeightReduce(G, W, V, 1, low, 100000000)
    return P

def partition_ideal(G, W, V_, node):
    I = set(get_ideal(G, V_, node)); N = len(V_)
    V = [0 if i in I else 1 for i in range(N)]

    R = get_size(G, V, node)
    low = 0; high = R
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppWeightReduce(G, W, V, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppWeightReduce(G, W, V, 1, low, 100000000)
    return P, get_ideal(G, V, P[0])

## greedyAppWeightReduce

- Input: Grafo (G), Lista de pesos (W), lista de nodos visitados (V_), Cantidad de nodos (N), Cantidad de muestras (K), límite de nodos en la solución (limit) y límite de personas en la solución (wlim)
- Output: Lista con las K soluciones, Diccionario de soluciones, tamano de las particiones inducidas, diccionario de particiones inducidas (nodo a testear: lista de nodos correspondientes a partición)

In [5]:
def greedyAppWeightReduceDynamic_partition(G, V_, N, K, limit, wlim):
    
    V = V_.copy() 
    E = {u for u in range(N) if not V[u]}


    ans = []; divided = []; solution_dict = {}; precalc = {}
    count = 0; tot = 0; sizes = {}; ideals = {}
    while count < K:
        
        S = [0 for u in range(N)]
        for v in E:
            _, S[v] = get_size_weight(G, V, N, v)
        
        maxv = 0; u = -1
        for v in E:
            s = get_size_weight_precalc(G, V, S, N, v)
            if s > maxv and s <= limit and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break
        
        moved = 0
        for node in list(set(ans) - set(divided)):
            ideal_size, ideal, s, ideal_partition  = precalc[node]; size_partition = len(ideal_partition) 
            min_partition = min(size_partition, ideal_size - size_partition)
            if min_partition > len(get_ideal(G, V, u)):
                u = s
                print('breaking ideal')
                divided.append(node); divided.append(u); moved = 1;
                solution_dict[node] = [u]
                sizes[u] = size_partition; sizes[node] = ideal_size - size_partition
                ideals[u] = ideal_partition
                ideals[node] = [i for i in ideals[node] if i not in ideal_partition]
                break
                
        ideal = get_ideal(G, V, u); ideal_size = len(ideal)       
        if not moved:
            E.remove(u)
            solution_dict[u] = []
            V_partition = [0 if i in get_ideal(G, V, u) else 1 for i in range(N)]
            s, ideal_partition = partition_ideal(G, V_partition, N, u)
            precalc[u] = (ideal_size, ideal, s[0], ideal_partition); sizes[u] = ideal_size
            ideals[u] = ideal
            
        V, aux = visit(G, V, u)
        ans.append(u); tot += aux
        count += 1
        
    ans_ = list(itertools.chain.from_iterable(([j,i] if j != [] else [i] for i,j in solution_dict.items())))
    ans = [i[0] if type(i) == list else i for i in ans_]
    sizes['out'] = len(G.nodes()) - sum(V)
    return ans, solution_dict, sizes, ideals

In [6]:
def get_dp_table(G, V, N, k, s, mult):

    order = get_ideal(G, V, s)[::-1]
    size = get_size(G, V, s)

    possibilities = {u: [] for u in order}
    next_dicts = {u: {} for u in order}

    for u in order:

        new_possibilities = {}; new_next = {}
        predecessors = [v for v in G.predecessors(u) if not V[v]]

        mul = [1 for v in predecessors]; combs = 1 if predecessors else 0
        for i in range(len(predecessors)):
            now = len(possibilities[predecessors[i]])
            for j in range(i):
                mul[j] *= now
            combs *= now
        
        for i in range(combs):

            indexes = [0 for v in predecessors]
            for j in range(len(predecessors)):
                indexes[j] = (i // mul[j])
                i %= mul[j]

            l = 0; size_left = 1; max_ideal = 0; min_ideal = N
            for j in range(len(predecessors)):
                pos = possibilities[predecessors[j]][indexes[j]]
                l += pos[0]
                size_left += pos[1]
                max_ideal = max(max_ideal, pos[2])
                min_ideal = min(min_ideal, pos[3])

            if l >= k:
                continue

            if new_possibilities.get((l, max_ideal // mult)):
                prev = new_possibilities[(l, max_ideal // mult)]
                if size_left < prev[0] or (size_left == prev[0] and min_ideal > prev[2]):
                    new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
                    new_next[(l, max_ideal // mult)] = indexes
            else:
                new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
                new_next[(l, max_ideal // mult)] = indexes
                
            if l < k:
                if new_possibilities.get((l + 1, max(size_left, max_ideal) // mult)):
                    prev = new_possibilities[(l + 1, max(size_left, max_ideal) // mult)]
                    if 0 < prev[0] or (0 == prev[0] and min(size_left, min_ideal) > prev[2]):
                        new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
                        new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes
                else:
                    new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
                    new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes

        if combs == 0:
            new_possibilities[(0, 0)] = (1, 0, N)
            new_next[(0, 0)] = []
            new_possibilities[(1, 0)] = (0, 1, 1)
            new_next[(1, 0)] = []
        
        final_possibilities = set()
        for item in new_possibilities.items():
            o, b = item;
            a = (o[0], b[0], b[1], b[2])
            final_possibilities.add(a)
            next_dicts[u][a] = new_next[o]

        possibilities[u] = list(final_possibilities)

    return possibilities, next_dicts

def get_sample_from_table(possibilities, next_dicts, V, k, s):
    
    mv = 1e10; opt_pos = None
    for pos in possibilities[s]:
        if pos[0] == k and pos[1] == 0:
            now = pos[2]
            if now < mv:
                mv = now
                opt_pos = pos
                
    ans = []
    Q = deque([]); Q.append((s, opt_pos))
    while Q:
        u, pos = Q.popleft()

        if pos[1] == 0:
            ans.append(u)

        i = 0; indexes = next_dicts[u][pos]

        predecessors = [v for v in G.predecessors(u) if not V[v]]
        
        for v in predecessors:
            next_pos_v = possibilities[v][indexes[i]]
            Q.append((v, next_pos_v)); i += 1

    N = len(V); V_ = [0] * N; sizes = []
    for u in ans[::-1]:
        V_, x = visit(G, V_, u)
        sizes.append(x)
    
    return ans, sizes